In [1]:
import pandas as pd
data_path = "../../data/open_source_8454_combine_short_description.csv"  # 替換為你的檔案路徑
df = pd.read_csv(data_path)
# df_label_0 = df[df["Label"] == 0]
# df_label_0_to_drop = df_label_0.sample(n=3000, random_state=42)
# df = df.drop(df_label_0_to_drop.index)
df

,Inbound Message,Label
0,login issue verified user details employee man...,0
1,outlook hello team meetings skype meetings etc...,0
2,cant log vpn cannot log vpn best,0
3,unable access tool page unable access tool page,0
4,skype error skype error,0
...,...,...
8449,emails coming mail good afternoon receiving em...,22
8450,telephony software issue telephony software issue,0
8451,vip windows password reset tifpdchb pedxruyf v...,0
8452,machine est funcionando unable access machine ...,44


# 十折驗證法

In [2]:
import torch
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print(f"Using device: {device}") 
print(f"CUDA Available: {torch.cuda.is_available()}")
print(f"CUDA Device: {torch.cuda.get_device_name(0)}")
print(f"Current GPU Memory Allocated: {torch.cuda.memory_allocated() / 1e9} GB")
print(f"Current GPU Memory Cached: {torch.cuda.memory_reserved() / 1e9} GB")

Using device: cuda
CUDA Available: True
CUDA Device: NVIDIA GeForce RTX 3050 6GB Laptop GPU
Current GPU Memory Allocated: 0.0 GB
Current GPU Memory Cached: 0.0 GB


In [3]:
from torch.utils.data import Dataset
class CustomDataset(Dataset):
    def __init__(self, messages, labels, tokenizer, max_length):
        self.messages = messages
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.messages)

    def __getitem__(self, idx):
        message = str(self.messages[idx])
        label = self.labels[idx]
        
        encoding = self.tokenizer.encode_plus(
            message,
            max_length=self.max_length,
            add_special_tokens=True,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt'
        )
        
        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': torch.tensor(label, dtype=torch.long)
        }

In [4]:
# 定義訓練與評估函數
def train_epoch(model, dataloader, optimizer, criterion):
    model.train()
    total_loss = 0

    for batch in tqdm(dataloader):
        optimizer.zero_grad()

        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        total_loss += loss.item()

        loss.backward()
        optimizer.step()
    
    return total_loss / len(dataloader)

In [5]:
from sklearn.metrics import precision_score, recall_score, f1_score, balanced_accuracy_score, matthews_corrcoef
from sklearn.preprocessing import label_binarize
import numpy as np
import torch

def eval_model(model, dataloader, criterion, num_labels):
    model.eval()
    total_loss = 0
    all_labels = []
    all_preds = []
    all_probs = []
    
    with torch.no_grad():
        for batch in dataloader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)

            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            logits = outputs.logits
            loss = criterion(logits, labels)
            total_loss += loss.item()

            probs = torch.softmax(logits, dim=1)
            _, preds = torch.max(logits, dim=1)

            all_labels.extend(labels.cpu().numpy())
            all_preds.extend(preds.cpu().numpy())
            all_probs.extend(probs.cpu().numpy())

    val_loss = total_loss / len(dataloader)
    val_accuracy = np.mean(np.array(all_preds) == np.array(all_labels))
    precision = precision_score(all_labels, all_preds, average='weighted', zero_division=0)
    recall = recall_score(all_labels, all_preds, average='weighted', zero_division=0)
    f1 = f1_score(all_labels, all_preds, average='weighted', zero_division=0)
    macro_f1 = f1_score(all_labels, all_preds, average='macro', zero_division=0)
    balanced_acc = balanced_accuracy_score(all_labels, all_preds)
    mcc = matthews_corrcoef(all_labels, all_preds)

    return {
        "val_loss": val_loss,
        "val_accuracy": val_accuracy,
        "precision": precision,
        "recall": recall,
        "f1-score": f1,
        "macro_f1-score": macro_f1,
        "balanced_accuracy": balanced_acc,
        "mcc": mcc
    }

In [6]:
from transformers import DebertaTokenizer, DebertaForSequenceClassification
from sklearn.model_selection import KFold
# 設定參數
MAX_LENGTH = 128
BATCH_SIZE = 16
EPOCHS = 20

# 初始化 BERT 的 tokenizer
tokenizer = DebertaTokenizer.from_pretrained('microsoft/deberta-base')

# 初始化數據
messages = df['Inbound Message'].tolist()
labels = df['Label'].tolist()

# KFold 初始化
kf = KFold(n_splits=10, shuffle=True, random_state=42)

In [7]:
from torch.utils.data import DataLoader
import torch
from torch.nn import CrossEntropyLoss
from torch.optim import AdamW
from tqdm import tqdm
import numpy as np

all_fold_results = []

# 進行 10 折交叉驗證
for fold, (train_idx, val_idx) in enumerate(kf.split(messages)):
    print(f"Fold {fold + 1}")
    
    train_messages = [messages[i] for i in train_idx]
    train_labels = [labels[i] for i in train_idx]
    val_messages = [messages[i] for i in val_idx]
    val_labels = [labels[i] for i in val_idx]

    print("Training set label distribution:", {label: train_labels.count(label) for label in set(train_labels)})
    print("Validation set label distribution:", {label: val_labels.count(label) for label in set(val_labels)})
    
    train_dataset = CustomDataset(train_messages, train_labels, tokenizer, MAX_LENGTH)
    val_dataset = CustomDataset(val_messages, val_labels, tokenizer, MAX_LENGTH)
    train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
    val_dataloader = DataLoader(val_dataset, batch_size=BATCH_SIZE)

    NUM_LABELS = len(set(train_labels) | set(val_labels))
    model = DebertaForSequenceClassification.from_pretrained('microsoft/deberta-base', num_labels=NUM_LABELS)
    model.to(device)
    optimizer = AdamW(model.parameters(), lr=5e-5, weight_decay=1e-2)
    criterion = CrossEntropyLoss()

    epoch_results = []
    patience = 3
    best_macro_f1 = 0
    epochs_without_improvement = 0
    
    for epoch in range(EPOCHS):
        print(f"Epoch {epoch + 1}/{EPOCHS}")
        train_loss = train_epoch(model, train_dataloader, optimizer, criterion)
        metrics = eval_model(model, val_dataloader, criterion, num_labels=NUM_LABELS)
        epoch_results.append(metrics)
        print(metrics)

        current_macro_f1 = metrics["macro_f1-score"]
        
        if current_macro_f1 > best_macro_f1:
            best_macro_f1 = current_macro_f1  # 更新最佳 macro_f1-score
            epochs_without_improvement = 0   # 重置計數
        else:
            epochs_without_improvement += 1  # 增加計數
        
        # 若連續 patience 3 次沒有提升，則停止訓練
        if epochs_without_improvement >= patience:
            print(f"Early stopping triggered at epoch {epoch + 1}")
            break
    
    avg_metrics = {metric: np.mean([epoch[metric] for epoch in epoch_results]) for metric in epoch_results[0].keys()}
    all_fold_results.append(avg_metrics)

final_metrics = {metric: np.mean([result[metric] for result in all_fold_results]) for metric in all_fold_results[0].keys()}
print("Final 10-fold Cross-Validation Results:")
for metric, value in final_metrics.items():
    print(f"{metric}: {value}")

Fold 1
Training set label distribution: {0: 3585, 1: 28, 2: 123, 3: 26, 4: 239, 5: 134, 6: 101, 7: 37, 8: 80, 9: 70, 10: 82, 11: 199, 12: 209, 13: 34, 14: 25, 15: 31, 16: 22, 17: 258, 18: 106, 19: 47, 20: 15, 21: 39, 22: 89, 23: 183, 24: 15, 25: 55, 26: 100, 27: 52, 28: 13, 29: 15, 30: 17, 31: 89, 32: 39, 33: 36, 34: 29, 35: 13, 36: 32, 37: 24, 38: 5, 39: 118, 40: 10, 41: 10, 42: 166, 43: 18, 44: 23, 45: 11, 46: 58, 47: 601, 48: 215, 49: 82}
Validation set label distribution: {0: 389, 1: 3, 2: 17, 3: 4, 4: 18, 5: 11, 6: 17, 7: 2, 8: 5, 9: 11, 10: 6, 11: 16, 12: 32, 13: 2, 14: 4, 16: 3, 17: 31, 18: 10, 19: 9, 20: 3, 21: 5, 22: 8, 23: 17, 24: 5, 25: 8, 26: 7, 27: 9, 28: 2, 29: 1, 30: 2, 31: 11, 32: 6, 33: 4, 34: 8, 35: 2, 36: 3, 37: 3, 38: 2, 39: 11, 40: 4, 41: 1, 42: 18, 43: 2, 44: 2, 46: 10, 47: 60, 48: 37, 49: 5}


Some weights of DebertaForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-base and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/20


100%|██████████| 476/476 [16:02<00:00,  2.02s/it]


{'val_loss': 1.5639162221044864, 'val_accuracy': 0.5898345153664303, 'precision': 0.5187904118898589, 'recall': 0.5898345153664303, 'f1-score': 0.5178299576052748, 'macro_f1-score': 0.13608323908096617, 'balanced_accuracy': 0.15465629242692108, 'mcc': 0.4409511431574888}
Epoch 2/20


...

{'val_loss': 1.356179009630995, 'val_accuracy': 0.6520710059171597, 'precision': 0.6374561562956005, 'recall': 0.6520710059171597, 'f1-score': 0.6202804383655391, 'macro_f1-score': 0.34430226182541906, 'balanced_accuracy': 0.3622065103754155, 'mcc': 0.5528275136617369}
Epoch 6/20


100%|██████████| 476/476 [14:40<00:00,  1.85s/it]


{'val_loss': 1.4836285895896408, 'val_accuracy': 0.6520710059171597, 'precision': 0.6773347908782245, 'recall': 0.6520710059171597, 'f1-score': 0.6364544852183246, 'macro_f1-score': 0.33371244607744777, 'balanced_accuracy': 0.3684859842719303, 'mcc': 0.5605081232599973}
Epoch 7/20


100%|██████████| 476/476 [14:40<00:00,  1.85s/it]


{'val_loss': 1.638375554444655, 'val_accuracy': 0.6603550295857988, 'precision': 0.6340685125134239, 'recall': 0.6603550295857988, 'f1-score': 0.6271372226517241, 'macro_f1-score': 0.33580187203630657, 'balanced_accuracy': 0.3165045783933259, 'mcc': 0.5560521419108335}
Early stopping triggered at epoch 7
Final 10-fold Cross-Validation Results:
val_loss: 1.557405379220508
val_accuracy: 0.6398660005509432
precision: 0.6028719619912024
recall: 0.6398660005509432
f1-score: 0.598084310626292
macro_f1-score: 0.27391204028318783
balanced_accuracy: 0.28915912886492995
mcc: 0.49226847592751444


In [8]:
all_fold_results

[{'val_loss': 1.5485965854793429,
  'val_accuracy': 0.6410917687513432,
  'precision': 0.6300185905085421,
  'recall': 0.6410917687513432,
  'f1-score': 0.6152048287938127,
  'macro_f1-score': 0.31850658483972966,
  'balanced_accuracy': 0.33304077240823465,
  'mcc': 0.5267703661375908},
 {'val_loss': 1.4564624436715112,
  'val_accuracy': 0.6718873128447597,
  'precision': 0.6594430945516592,
  'recall': 0.6718873128447597,
  'f1-score': 0.6422401171763679,
  'macro_f1-score': 0.300894650679114,
  'balanced_accuracy': 0.31240673452488743,
  'mcc': 0.5536186604990123},
 {'val_loss': 1.4136892903450178,
  'val_accuracy': 0.6861247499545372,
  'precision': 0.6726110691543451,
  'recall': 0.6861247499545372,
  'f1-score': 0.6629383719617762,
  'macro_f1-score': 0.32620881386214634,
  'balanced_accuracy': 0.3359553083530912,
  'mcc': 0.5647675232012543},
 {'val_loss': 1.3597347851280333,
  'val_accuracy': 0.6696217494089834,
  'precision': 0.6494979976386359,
  'recall': 0.6696217494089834,
